In [1]:
import pandas as pd

In [2]:
res = pd.read_csv('../data/geo-annotation/age_annotation_result.csv', sep='\t')

In [3]:
import seaborn
%matplotlib inline

In [4]:
res.query('age > 120')[:4]

,rid,title,accession,description,age_str,age,detected,detect_crit,is_form_body,is_form_cell_line,datetime,source_name
11780,604503,hK4,GSM1198468,Sample 10|mRNA|Processed data file: 3_species_...,334,334,1,7,1,0,2015-10-01 02:51:09,kidney
11783,604506,hP1,GSM1198471,Sample 13|mRNA|Processed data file: 3_species_...,341,341,1,7,1,0,2015-10-01 02:51:09,PFC
11795,604518,hC1,GSM1198483,Sample 25|mRNA|Processed data file: 3_species_...,341,341,1,7,1,0,2015-10-01 02:51:10,CBC
12116,608072,SEQC NB patient 001,GSM1205238,NaN,987,987,1,7,1,0,2015-10-01 02:52:20,neuroblastoma


In [5]:
def age_name(age_str):
    tokens = age_str.split(' ')
    if len(tokens) == 2:
        return tokens[1]
    else:
        if len(tokens) == 1:
            return '[=1]'
        else: 
            return '[>2]'

In [6]:
print(res.shape)
res.age_str.map(age_name).value_counts()

(89059, 12)


[=1]              81369
years              3228
[>2]               2226
y                   502
yrs                 461
yr                  351
days                277
weeks               199
months              151
years;               70
year                 47
wk                   31
yo                   24
yrs.                 23
y.o.                 22
year-old             20
y/o                  10
month                 8
day                   6
11m                   5
3m                    4
5m                    3
7m                    3
9m                    3
week                  3
differentiated        2
22d                   1
(14d)                 1
4m                    1
2m                    1
(6d)                  1
m                     1
mo                    1
yesrs                 1
8m                    1
years%                1
1m                    1
dtype: int64

In [7]:
?pd.cut

In [8]:
res.query('age < 0 or age > 100').shape[0]

1021

In [9]:
res.shape[0]

89059

In [10]:
res.query('0 < age < 100').shape[0]

87888

In [11]:
res.groupby(pd.cut(res.age, bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])).age.count().to_frame()

,age
age,
"(0, 10]",7030
"(10, 20]",4856
"(20, 30]",6211
"(30, 40]",8932
"(40, 50]",13886
"(50, 60]",17719
"(60, 70]",16334
"(70, 80]",9316
"(80, 90]",3089


In [12]:
import pymongo
db = pymongo.MongoClient().scraper_meta

In [13]:
platforms = (
    pd.DataFrame(list(db.samples.find(
            {'accession': {'$in': res.accession.tolist()}},
            {'_id': 0, 'accession' :1, 'platform': 1})))
)
platforms.head()

,accession,platform
0,GSM15693,GPL96
1,GSM15694,GPL96
2,GSM15722,GPL96
3,GSM15723,GPL96
4,GSM15724,GPL96


In [14]:
res_platforms = pd.merge(res, platforms)
res_platforms.head()

,rid,title,accession,description,age_str,age,detected,detect_crit,is_form_body,is_form_cell_line,datetime,source_name,platform
0,513737,Case 8 LNM1,GSM1000061,Twelve non-neoplastic kidney tissues mix was u...,62,62,1,7,1,0,2015-10-01 02:20:03,OSCC neck lymph-node metastasis,GPL5477
1,513737,Case 8 LNM1,GSM1000061,Twelve non-neoplastic kidney tissues mix was u...,62,62,1,7,1,0,2015-10-01 02:20:03,OSCC neck lymph-node metastasis,GPL5477
2,513738,Case 8 LNM2,GSM1000062,Twelve non-neoplastic kidney tissues mix was u...,62,62,1,7,1,0,2015-10-01 02:20:03,OSCC neck lymph-node metastasis,GPL5477
3,513738,Case 8 LNM2,GSM1000062,Twelve non-neoplastic kidney tissues mix was u...,62,62,1,7,1,0,2015-10-01 02:20:03,OSCC neck lymph-node metastasis,GPL5477
4,513739,cancer preop 1,GSM1000223,files with prefix 1_ are for Hy3 (Cy3) signals...,41 y,41,1,7,1,0,2015-10-01 02:20:03,"blood, lung cancer, individual 1",GPL16016


In [15]:
(
    res_platforms
    .query(' 0< age<100')
    .platform
    .value_counts()
    .to_frame()
    .rename(columns={0: 'count'})
    .query('count > 200')
    [:20]
)

,count
GPL570,12166
GPL13534,9494
GPL96,4785
GPL8490,4352
GPL10558,3834
GPL6244,3807
GPL6947,3346
GPL4133,2082
GPL6480,1719
GPL9183,1502


In [17]:
_c = db.samples.find({'accession': {'$in': res.accession.tolist()}},
                         {'_id': 0, 'accession' :1, 'series': 1})

series_samples = pd.DataFrame.from_records([(x['accession'], series) for x in _c for series in x['series']],
                                           columns=['accession', 'series'])


In [19]:
series_samples = series_samples.drop_duplicates()

In [21]:
res_series = pd.merge(res, series_samples)

In [22]:
res_series.to_pickle('../data/geo-annotation/series.age.res.1.py3.pickle')